## 목표
결정 트리를 사용해 moons 데이터셋 분류 문제를 해결하고, 랜덤 포레스트로 정확도를 향상시켜 보세요.

### 1. 데이터 준비
- `make_moons(n_samples=10000, noise=0.4)`를 사용해 데이터를 생성하세요.
- 데이터를 훈련 세트(80%)와 테스트 세트(20%)로 나누세요(`train_test_split` 사용).

In [9]:
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

# 데이터 생성
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

# 훈련 세트와 테스트 세트로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 2. 결정 트리 하이퍼파라미터 최적화
- `GridSearchCV`를 사용해 결정 트리의 `max_depth`, `max_leaf_nodes` 등 최적의 하이퍼파라미터를 찾으세요.
- 최적의 하이퍼파라미터로 훈련된 모델의 테스트 정확도를 확인하세요.


In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 결정 트리 모델 설정
tree_clf = DecisionTreeClassifier(random_state=42)

# 하이퍼파라미터 그리드 설정
param_grid = {
    'max_depth': [3, 4, 5, 6, 7, 8],
    'max_leaf_nodes': [10, 15, 20, 25, 30],
    'min_samples_split': [2, 5, 10]
}

# GridSearchCV 실행
grid_search = GridSearchCV(tree_clf, param_grid, cv=5, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# 최적의 모델 테스트 정확도 확인
best_tree = grid_search.best_estimator_
test_accuracy = best_tree.score(X_test, y_test)
print("Single Tree Accuracy:", test_accuracy)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best Parameters: {'max_depth': 8, 'max_leaf_nodes': 25, 'min_samples_split': 2}
Single Tree Accuracy: 0.872


### 3. 랜덤 포레스트 구현
- 훈련 세트를 랜덤하게 샘플링한 서브셋을 생성하세요(100개의 서브셋).
- 각 서브셋에 대해 결정 트리를 훈련시키고 정확도를 확인하세요.

In [11]:
from sklearn.utils import resample
from sklearn.metrics import accuracy_score

n_trees = 100  # 서브셋 개수
subsets = []   # 서브셋 저장 리스트
trees = []     # 학습된 결정 트리 저장

# 랜덤 서브셋 생성 및 결정 트리 학습
for i in range(n_trees):
    X_subset, y_subset = resample(X_train, y_train, random_state=i)
    tree = DecisionTreeClassifier(**best_params, random_state=i)
    tree.fit(X_subset, y_subset)
    subsets.append((X_subset, y_subset))
    trees.append(tree)

# 각 트리 정확도 저장 리스트
tree_accuracies = []

# 각 트리의 예측값과 정확도 확인
predictions = np.zeros((n_trees, len(X_test)))

for idx, tree in enumerate(trees):
    y_pred = tree.predict(X_test)
    predictions[idx] = y_pred

    # 각 트리 정확도 저장
    acc = accuracy_score(y_test, y_pred)
    tree_accuracies.append(acc)

# 단일 트리 평균 정확도 계산
avg_single_tree_accuracy = np.mean(tree_accuracies)
print(f"Average Single Tree Accuracy: {avg_single_tree_accuracy:.4f}")

Average Single Tree Accuracy: 0.8634


### 4. 다수결 앙상블
- 테스트 데이터에 대해 100개의 결정 트리의 예측값을 모으세요.
- 다수결 방식으로 최종 예측을 생성하고 정확도를 측정하세요.

In [12]:
from scipy.stats import mode

# 각 트리의 예측값 모으기
predictions = np.zeros((n_trees, len(X_test)))

for idx, tree in enumerate(trees):
    predictions[idx] = tree.predict(X_test)

# 다수결 앙상블
ensemble_predictions, _ = mode(predictions, axis=0)
ensemble_predictions = ensemble_predictions.ravel()

# 앙상블 정확도 확인
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print("Ensemble Accuracy:", ensemble_accuracy)


Ensemble Accuracy: 0.8645
